In [ ]:
import sys

In [ ]:
sys.path

In [ ]:
sys.path.append("/home/svu/e0384946/anaconda3/lib/python3.7/site-packages")

In [ ]:
%cd Pedestrian-Attribute-Recognition

In [ ]:
import os
import pprint
from collections import OrderedDict, defaultdict

import numpy as np
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader

from batch_engine import valid_trainer, batch_trainer
from config import argument_parser
from dataset.AttrDataset import AttrDataset, get_transform
from loss.CE_loss import CEL_Sigmoid
from models.base_block import FeatClassifier, BaseClassifier
from models.resnet import resnet50
from tools.function import get_model_log_path, get_pedestrian_metrics
from tools.utils import time_str, save_ckpt, ReDirectSTD, set_seed

import getpass
import inspect

set_seed(605)

In [ ]:
def trainer(epoch, model, train_loader, valid_loader, criterion, optimizer, lr_scheduler,
            path):
    maximum = float(-np.inf)
    best_epoch = 0

    result_list = defaultdict()

    for i in range(epoch):

        train_loss, train_gt, train_probs = batch_trainer(
            epoch=i,
            model=model,
            train_loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
        )

        valid_loss, valid_gt, valid_probs = valid_trainer(
            model=model,
            valid_loader=valid_loader,
            criterion=criterion,
        )

        lr_scheduler.step(metrics=valid_loss, epoch=i)

        train_result = get_pedestrian_metrics(train_gt, train_probs)
        valid_result = get_pedestrian_metrics(valid_gt, valid_probs)

        print(f'Evaluation on test set, \n',
              'ma: {:.4f},  pos_recall: {:.4f} , neg_recall: {:.4f} \n'.format(
                  valid_result.ma, np.mean(valid_result.label_pos_recall), np.mean(valid_result.label_neg_recall)),
              'Acc: {:.4f}, Prec: {:.4f}, Rec: {:.4f}, F1: {:.4f}'.format(
                  valid_result.instance_acc, valid_result.instance_prec, valid_result.instance_recall,
                  valid_result.instance_f1))

        print(f'{time_str()}')
        print('-' * 60)

        cur_metric = valid_result.ma

        if cur_metric > maximum:
            maximum = cur_metric
            best_epoch = i
            save_ckpt(model, path, i, maximum)

        result_list[i] = [train_result, valid_result]

    torch.save(result_list, os.path.join(os.path.dirname(path), 'metric_log.pkl'))

    return maximum, best_epoch

In [ ]:
class args_config():
    def __init__(self):
        self.dataset="RAP"
        self.debug='store_false'
        self.batchsize=64
        self.train_epoch=30
        self.height=256
        self.width=192
        self.lr_ft=0.01
        self.lr_new=0.1
        self.classifier='base'
        self.momentum=0.9
        self.weight_decay=5e-4
        self.train_split="trainval" #choices=['train' 'trainval']
        self.valid_split="test" #choices=['test' 'valid']
        self.device=0
        self.redirector='store_false'
        self.use_bn='store_false'

In [ ]:
args = args_config()

In [ ]:
# args.dataset = "PA100k"
args.dataset = "PETA"

In [ ]:
visenv_name = args.dataset
exp_dir = os.path.join('exp_result', args.dataset)
model_dir, log_dir = get_model_log_path(exp_dir, visenv_name)

# Added for logging purposes
user = getpass.getuser() 
fixed_time_str = time_str()
stdout_file = os.path.join(log_dir, "_".join(['stdout', user, f'{fixed_time_str}.txt']) )
save_model_path = os.path.join(model_dir,  "_".join(['ckpt_max', user, f'{fixed_time_str}.pth']) )
trackitems_dir = os.path.join(log_dir, "_".join(['trackitems', user, f'{fixed_time_str}.txt']) )


if args.redirector:
    print('redirector stdout')
    ReDirectSTD(stdout_file, 'stdout', False)

pprint.pprint(OrderedDict(args.__dict__))

print('-' * 60)
print(f'use GPU{args.device} for training')
print(f'train set: {args.dataset} {args.train_split}, test set: {args.valid_split}')

train_tsfm, valid_tsfm = get_transform(args)
print(train_tsfm)

train_set = AttrDataset(args=args, split=args.train_split, transform=train_tsfm)

train_loader = DataLoader(
    dataset=train_set,
    batch_size=args.batchsize,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)
valid_set = AttrDataset(args=args, split=args.valid_split, transform=valid_tsfm)

valid_loader = DataLoader(
    dataset=valid_set,
    batch_size=args.batchsize,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
)

print(f'{args.train_split} set: {len(train_loader.dataset)}, '
      f'{args.valid_split} set: {len(valid_loader.dataset)}, '
      f'attr_num : {train_set.attr_num}')

labels = train_set.label
sample_weight = labels.mean(0)

backbone = resnet50()
classifier = BaseClassifier(nattr=train_set.attr_num)
model = FeatClassifier(backbone, classifier)

if torch.cuda.is_available():
    model = torch.nn.DataParallel(model).cuda()

criterion = CEL_Sigmoid(sample_weight)

if torch.cuda.is_available():
    param_groups = [{'params': model.module.finetune_params(), 'lr': args.lr_ft},
                    {'params': model.module.fresh_params(), 'lr': args.lr_new}]
else:
    param_groups = [{'params': model.finetune_params(), 'lr': args.lr_ft},
                    {'params': model.fresh_params(), 'lr': args.lr_new}]
optimizer = torch.optim.SGD(param_groups, momentum=args.momentum, weight_decay=args.weight_decay, nesterov=False)
lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=4)

# Added for logging purposes
with open(trackitems_dir, "a") as f:
    code, line_no = inspect.getsourcelines(get_transform)
    for line in code:
        f.write(str(line))
    f.write(str("\n\n"))

    f.write(str(args.__dict__))
    f.write(str("\n\n"))

    f.write(str(lr_scheduler.__dict__))
    f.write(str("\n\n"))

    model_str = str(model).lower()
    have_dropout = 'dropout' in model_str
    f.write('dropout: %s' %(have_dropout))
    f.write(str("\n\n"))

    have_leaky_relu = 'leaky_relu' in model_str
    f.write('leaky_relu: %s' %(have_leaky_relu))
    f.write(str("\n\n"))

In [ ]:
best_metric, epoch = trainer(epoch=args.train_epoch,
                             model=model,
                             train_loader=train_loader,
                             valid_loader=valid_loader,
                             criterion=criterion,
                             optimizer=optimizer,
                             lr_scheduler=lr_scheduler,
                             path=save_model_path)

print(f'{visenv_name},  best_metrc : {best_metric} in epoch{epoch}')

# Added for logging purposes
with open(trackitems_dir, "a") as f:
    f.write(f'{visenv_name},  best_metrc : {best_metric} in epoch{epoch}')
    f.write("\n\n")
